In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

url_list = "http://www.btbtdy.com/screen/1--2018---time-1.html"

driver = webdriver.PhantomJS('D:\\Github\\MovieCrawler\\phantomjs.exe')
# get()方法會一直等到頁面被完全加載，然後才會繼續程序
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
#print (soup.prettify())
# p_list = soup.find_all('p', class_='title')

# for p in p_list:
#     print(p.contents[0].text)


# a_list = soup.find_all('a', target='_blank')
# for a in a_list:
#     print(a)
#     print("a.content is %s" % (a.contents))  

# a_list = soup.find_all('a', target='_blank')
# for a in a_list:
#     print(a)
#     print("a.content is %s" % (a.contents))  



# 打印獲取的文本內容
#print(data)

# 打印頁面標題：XXXX年最新电影_第1页-BT电影天堂
print(driver.title)

p_list = soup.find_all('p', class_='title')
f = open('MovieNames.txt', 'w', encoding = 'UTF-8')    # 也可使用指定路徑等方式，如： C:\A.txt
for p in p_list:
    f.write(p.contents[0].text + '\n')

f.close()    
    
driver.quit()

# http://www.btbtdy.com/ herf=btdy/dy11737.html

2018年最新电影_第1页-BT电影天堂


## 讀取txt檔案

In [11]:
movieNameList=[]
f = open("D:\\Github\\MovieCrawler\\MovieNames.txt","r", encoding = 'UTF-8')
for line in f:
    #print(line[:-1])
    movieNameList.append(line[:-1])
#In [13]:l = list(range(10))
#In [13]: l
#Out[13]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#In [16]: l[:-1]
#Out[16]: [0, 1, 2, 3, 4, 5, 6, 7, 8]

f.close() 
# for i in movieNameList:
#     print(i)
print (movieNameList[0:6])


['末栈之黑水', '末栈之怪笔新娘', 'S4侠降魔记', '移动迷宫3：死亡解药', '奇迹男孩', '大世界']


## 爬Imdb頁面

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

url_list = "http://www.imdb.com/title/tt5556068/"

driver = webdriver.PhantomJS('D:\\Github\\MovieCrawler\\phantomjs.exe')
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

d_list = soup.find_all('div', class_='ratingValue')

print(d_list)
# for d in d_list:
#     print(d.descendants)

driver.quit()

### 實際測試
#### 為了不要一直import , 先import一次

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
from multiprocessing import Pool
import os

In [21]:
%%time
totalList = []

def getDobanAndImdbRating(movieName):
    doubanUrl_list = "https://movie.douban.com/"
    driver = webdriver.PhantomJS('D:\\Github\\MovieCrawler\\phantomjs.exe')
    driver.get(doubanUrl_list)
    outputList=[]
    
    # 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    #找豆瓣電影的search bar
    inputSearch = driver.find_element_by_name("search_text")
    ChineseMovieName = movieName
    inputSearch.send_keys(ChineseMovieName)
    inputSearch.submit()
    #time.sleep(2)

    #印出找到的電影名稱跟豆瓣評分
    searchOutput = driver.find_element_by_class_name("title")
    outputList.append(ChineseMovieName)
    searchHerf = searchOutput.find_element_by_css_selector('a').get_attribute('href')
    outputList.append(searchHerf)
    rating_num = driver.find_element_by_class_name("rating_nums")
    outputList.append("豆瓣 : "+rating_num.text)
    
    #點進去該電影後找Imdb的連結
    driver.get(searchHerf)
    # get()方法會一直等到頁面被完全加載，然後才會繼續程序, 所以不需要sleep等待
    html = driver.page_source
    try:
        IMDBLink = driver.find_element_by_partial_link_text("tt").get_attribute('href')
    except:
        print ("There is no IMDB!")
        outputList.append("沒有Imdb 連結")
        return outputList
    outputList.append("Imdb 連結: " +IMDBLink)
    driver.get(IMDBLink)
    #html = driver.page_source
    #soup = BeautifulSoup(html, "html.parser")
    try:
        ImdbRating = driver.find_element_by_class_name("ratingValue")
    except:
        print ("There is no IMDB rating!")
        outputList.append("沒有Imdb rating")
        return outputList
    outputList.append("IMDB : " +ImdbRating.text)
    imdbMovieName = driver.find_element_by_xpath("//*[@id='title-overview-widget']//h1")
    outputList.append(imdbMovieName.text)
    
    return outputList

# for i in movieNameList:
#     totalList.append(getDobanAndImdbRating(i))
#     print(totalList)
#     print("==============================================================================")
totalList.append(getDobanAndImdbRating(movieNameList[5]))
print(totalList)

# p = Pool(4) # 设置进程池大小
# for i in movieNameList[0:4]:
#     p.apply_async(getDobanAndImdbRating, args=(i,)) # 设置每个进程要执行的函数和参数
# print('Waiting for all subprocesses done...')
# p.close()
# p.join()
# print('All subprocesses done.')
        

[['母亲！', 'https://movie.douban.com/subject/26648284/', '豆瓣 : 6.6', 'Imdb 連結: http://www.imdb.com/title/tt5109784', 'IMDB : 6.8/10', 'Mother! (2017)']]
Wall time: 1min 8s


## 整理code如下


In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup

url_list = "https://movie.douban.com/"

driver = webdriver.PhantomJS('D:\\Github\\MovieCrawler\\phantomjs.exe')
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

#找豆瓣電影的search bar
inputSearch = driver.find_element_by_name("search_text")

#這邊輸入你要的中文電影名稱 EX: ChineseMovieName = u"挖崊老ㄙ"
#ChineseMovieName = u"天才槍手"

##改成用txt黨輸入要找的電影名稱
movieNameList=[]
f = open("D:\\Github\\MovieCrawler\\MovieNames.txt","r", encoding = 'UTF-8')
for line in f:
    movieNameList.append(line[:-1])

f.close()

ChineseMovieName = movieNameList[4]


inputSearch.send_keys(ChineseMovieName)
inputSearch.submit()
time.sleep(2)

#印出找到的電影名稱跟豆瓣評分
searchOutput = driver.find_element_by_class_name("title")
print(searchOutput.text)
searchHerf = searchOutput.find_element_by_css_selector('a').get_attribute('href')
print(searchHerf)
rating_num = driver.find_element_by_class_name("rating_nums")
print("豆瓣 : "+rating_num.text)

#點進去該電影後找Imdb的連結
driver.get(searchHerf)
# get()方法會一直等到頁面被完全加載，然後才會繼續程序, 所以不需要sleep等待
#time.sleep(3)
html = driver.page_source
IMDBLink = driver.find_element_by_partial_link_text("tt").get_attribute('href')
#IMDBLink = IMDBLink.find_element_by_css_selector('a').get_attribute('href')
#//span[contains(@class, 'pl')]
#//a[contains(text(), 'tt')]/@href
#//a[contains(@href, 'imdb')]
print(IMDBLink)
driver.get(IMDBLink)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


奇迹男孩 Wonder (2017)
https://movie.douban.com/subject/26787574/
豆瓣 : 8.6
http://www.imdb.com/title/tt2543472


In [23]:
ImdbRating = driver.find_element_by_class_name("ratingValue")
print("IMDB : " +ImdbRating.text)
movieName = driver.find_element_by_class_name("title_wrapper")
print(movieName.text)
imdbMovieName = driver.find_element_by_xpath("//*[@id='title-overview-widget']//h1")
print(imdbMovieName.text)
#//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[2]/div[2]/h1/text()
#"//*[@id='UserContent']//span[@itemprop='description']"


#driver.quit()

IMDB : 8.1/10
Wonder (2017)
GP | 1h 53min | Drama, Family | 1 December 2017 (Taiwan)
Wonder (2017)


## 因為不知道如何filter掉 h1後方的span, 乾脆用soup把所有span拔掉

In [24]:
while len(soup.find_all('span')) > 0:
    soup.span.extract()
imdbMovieName2 = soup.find('h1', itemprop="name").getText()
print(imdbMovieName2)

Wonder  


## 將得到的英文電影名稱丟到Rotten Tomato裡面查評價

In [25]:
RottenTomatoURL = "https://www.rottentomatoes.com/"
RTpage = driver.get(RottenTomatoURL)


In [26]:
print(imdbMovieName2)
RTsearchURL = "https://www.rottentomatoes.com/search/?search="
RTSearch = driver.get(RTsearchURL+imdbMovieName2)
# get()方法會一直等到頁面被完全加載，然後才會繼續程序, 所以不需要sleep等待


Wonder  


In [27]:
Score = driver.find_element_by_class_name("tmeter")
###//span[contains(@class, 'tMeterScore')]
print(Score.text)

NoSuchElementException: Message: {"errorMessage":"Unable to find element with class name 'tmeter'","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"95","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:62513","User-Agent":"Python http auth"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"class name\", \"value\": \"tmeter\", \"sessionId\": \"ad6010e0-058a-11e8-b0ce-5771806388b3\"}","url":"/element","urlParsed":{"anchor":"","query":"","file":"element","directory":"/","path":"/element","relative":"/element","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/element","queryKey":{},"chunks":["element"]},"urlOriginal":"/session/ad6010e0-058a-11e8-b0ce-5771806388b3/element"}}
Screenshot: available via screen
